In [1]:
import os
####*IMPORANT*: Have to do this line *before* importing tensorflow
os.environ['PYTHONHASHSEED']=str(1)

In [17]:
import tensorflow
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import glob
import os
import imgaug as ia
import random

from pathlib import Path
from numpy import asarray
from imgaug import augmenters as iaa
from tensorflow import keras
from keras import backend as K
from keras import layers
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
from skimage.io import imread
from skimage import exposure, color
from skimage.transform import resize

In [18]:
train_data_path = '../../sensix_daninhas/dataset/train'
test_data_path  = '../../sensix_daninhas/dataset/validation'

img_rows = 100
img_cols = 100
epochs = 200
batch_size = 20
n_channels = 3

In [19]:
num_of_train_samples = len(glob.glob(train_data_path + "/**/*" + ".jpg" , recursive=True))
num_of_test_samples  = len(glob.glob(test_data_path + "/**/*" + ".jpg"  , recursive=True))
print("Train Images : ", num_of_train_samples)
print("Test Images  : ", num_of_test_samples)

Train Images :  2470
Test Images  :  707


In [12]:
def prepare_dataset_func_1(img):
    '''
    img: array que representa a imagem que será modificada:
    
    description: Representa efeitos relacionados a coloração da imagem. Nenhuma
    rotação ou aproximação (zoom) é dado. Apenas possui filtros que alteram as 
    caracteristicas gerais das imagens
    
    return: array
    O array retornado representa a imagem modificada
    '''
    img = img.astype('uint8')
    if random_func(percentage = 10):
        seq = iaa.Sequential([
            #iaa.Affine(rotate=(0, 90)),
            #iaa.Crop(percent=(0.1, 0.2)),
            iaa.LinearContrast((1.0, 1.9)),
            iaa.Multiply((0.8, 1.2), per_channel=0.2),
            iaa.AdditiveGaussianNoise(scale=(10, 20)),
            #iaa.Crop(percent=(0, 0.2)),
            iaa.AddToBrightness((-30, 30)),
            iaa.AddToHue((-50, 50)),
            #iaa.flip.Fliplr(0.5)
            #iaa.AllChannelsHistogramEqualization()
        ],
        random_order=True)
        images_aug = seq(image=img)
    else:
        images_aug = img
    
    images_aug = np.expand_dims(images_aug, axis=0)
    images_aug = images_aug.astype('float32')
    return (images_aug)

In [42]:
def reset_random_seeds():
    os.environ['PYTHONHASHSEED']=str(1)
    tf.random.set_seed(1)
    np.random.seed(1)
    random.seed(1)

In [53]:
train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   #preprocessing_function=prepare_dataset_func_1,
                                   #fill_mode='constant',
                                   )

test_datagen = ImageDataGenerator(rescale=1. / 255,
                                  )


train_generator = train_datagen.flow_from_directory(train_data_path,
                                                    target_size=(img_rows, img_cols),
                                                    batch_size=batch_size,
                                                    class_mode='binary',
                                                    )

validation_generator = test_datagen.flow_from_directory(test_data_path,
                                                        target_size=(img_rows, img_cols),
                                                        batch_size=batch_size,
                                                        class_mode='binary',
                                                        )

Found 2925 images belonging to 2 classes.
Found 820 images belonging to 2 classes.


In [54]:
reset_random_seeds()

visible = tf.keras.layers.Input(shape=(img_rows, img_cols, n_channels))
conv1 = tf.keras.layers.Conv2D(8, (3,3), activation='relu')(visible)
pooling1 = tf.keras.layers.MaxPooling2D(2, 2)(conv1)
conv2 = tf.keras.layers.Conv2D(16, (3,3), activation='relu')(pooling1)
pooling2 = tf.keras.layers.MaxPooling2D(2,2)(conv2)
conv3 = tf.keras.layers.Conv2D(32, (3,3), activation='relu')(pooling2)
pooling3 = tf.keras.layers.MaxPooling2D(2,2)(conv3)
flat = tf.keras.layers.Flatten()(pooling3)
hidden1 = tf.keras.layers.Dense(64, activation='relu')(flat)
out = tf.keras.layers.Dense(1, activation='sigmoid')(hidden1)

model = tf.keras.Model(inputs=visible, outputs=out)
model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 98, 98, 8)         224       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 49, 49, 8)         0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 47, 47, 16)        1168      
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 23, 23, 16)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 21, 21, 32)        4640      
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 10, 10, 32)        0  

In [55]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#callbacks = CallbackStop()

history = model.fit(train_generator,
                    steps_per_epoch= num_of_train_samples // batch_size,
                    epochs=2,
                    validation_data=validation_generator,
                    validation_steps=num_of_test_samples // batch_size,
                    #callbacks=[callbacks]
                    )

Epoch 1/2
123/123 [==============================] - 9s 70ms/step - loss: 0.3130 - accuracy: 0.8764 - val_loss: 0.2320 - val_accuracy: 0.8529
Epoch 2/2
123/123 [==============================] - 8s 69ms/step - loss: 0.1076 - accuracy: 0.9550 - val_loss: 0.2399 - val_accuracy: 0.8543
